# Análise de dados de uma rede varejista

## Engajamento em campanhas
### Quais perfis de clientes respondem melhor às campanhas convertendo em compra?

## Importando as bibliotecas


In [ ]:
## Importando as bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##Importando os dados do MYDRIVE
Carregando o conjunto de dados original e fazemos uma primeira inspeção para entender sua estrutura.

In [ ]:
##importando do MYDRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##Lendoo a a basa de dados que será usada
df_cadastral = pd.read_csv('/content/drive/MyDrive/Base de Dados/CADASTRAL_0c88a2ae15.csv')
df_campanhas = pd.read_csv('/content/drive/MyDrive/Base de Dados/CAMPANHAS_1e7228c5ad.csv')
df_compras = pd.read_csv('/content/drive/MyDrive/Base de Dados/COMPRAS_865e1d278d.csv')

##Visualizando os dados

In [ ]:
# Verificar as primeiras linhas de cada base
display(df_cadastral.head())
display(df_campanhas.head())
display(df_compras.head())

# Verificar info e tipos
df_cadastral.info()
df_campanhas.info()
df_compras.info()



,cliente_id,sexo,idade,faixa_etaria,estado,cidade,escolaridade,profissao,estado_civil,renda_mensal,score_inicial,tipo_cliente,n_dependentes,canal_captacao,status_cadastro
0,C000000,Feminino,23,18-24,PE,Olinda,Superior,Comerciante,Casado(a),3507.09,475,Potencial,0,Indicação,Inativo
1,C000001,Masculino,22,18-24,PE,Outros,Pós-graduação,Auxiliar de Serviços Gerais,Solteiro(a),2831.55,584,Tradicional,2,Loja física,Bloqueado
2,C000002,Masculino,18,18-24,PE,Garanhuns,Fundamental,Autônomo,Casado(a),1500.00,300,Potencial,2,Indicação,Inativo
3,C000003,Masculino,37,35-44,PE,Paulista,Fundamental,Comerciante,Solteiro(a),2085.33,338,Tradicional,3,Loja física,Inativo
4,C000004,Masculino,74,60+,PE,Recife,Médio,Vendedor(a),Casado(a),1500.00,562,Potencial,2,Indicação,Bloqueado


,cliente_id,data_envio,tema_campanha,canal,abriu,clicou,converteu_em_compra
0,C039242,2025-03-25,Compre e Ganhe,WhatsApp,Sim,Sim,Não
1,C118819,2023-10-19,Cashback,SMS,Não,Não,Não
2,C114009,2024-04-01,Cashback,SMS,Sim,Não,Sim
3,C101599,2024-03-21,Desconto Exclusivo,Email,Sim,Sim,Sim
4,C088611,2024-12-27,Novo Produto,WhatsApp,Sim,Não,Não


,cliente_id,data_compra,valor_compra,categoria,canal_compra,status_entrega,forma_pagamento,parcelado,quantidade_parcelas
0,C121958,2024-09-25,169.17,Alimentos,Presencial,Entregue,Pix,Sim,4
1,C146867,2024-04-29,73.34,Higiene Pessoal,Presencial,Entregue,Débito,Sim,2
2,C131932,2023-09-11,151.52,Bebidas,Online,Entregue,Pix,Não,1
3,C103694,2024-01-04,626.98,Utilidades Domésticas,Online,Entregue,Pix,Sim,4
4,C119879,2024-11-04,524.60,Utilidades Domésticas,Presencial,Entregue,Boleto,Sim,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   cliente_id       150000 non-null  object 
 1   sexo             150000 non-null  object 
 2   idade            150000 non-null  int64  
 3   faixa_etaria     150000 non-null  object 
 4   estado           150000 non-null  object 
 5   cidade           150000 non-null  object 
 6   escolaridade     150000 non-null  object 
 7   profissao        150000 non-null  object 
 8   estado_civil     150000 non-null  object 
 9   renda_mensal     150000 non-null  float64
 10  score_inicial    150000 non-null  int64  
 11  tipo_cliente     150000 non-null  object 
 12  n_dependentes    150000 non-null  int64  
 13  canal_captacao   150000 non-null  object 
 14  status_cadastro  150000 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 17.2+ MB
<class 'pandas.core.frame.

##Verificando e tratando Valores ausentes (caso  exista)

In [ ]:
# Verificando valores ausentes na base
df_cadastral.isnull().sum().sort_values(ascending=False)

,0
cliente_id,0
sexo,0
idade,0
faixa_etaria,0
estado,0
cidade,0
escolaridade,0
profissao,0
estado_civil,0
renda_mensal,0


In [ ]:
df_campanhas.isnull().sum().sort_values(ascending=False)

,0
tema_campanha,500
clicou,500
cliente_id,0
data_envio,0
canal,0
abriu,0
converteu_em_compra,0


In [ ]:
df_compras.isnull().sum().sort_values(ascending=False)

,0
forma_pagamento,700
cliente_id,0
data_compra,0
categoria,0
valor_compra,0
canal_compra,0
status_entrega,0
parcelado,0
quantidade_parcelas,0


In [ ]:
# Tratamento de valores ausentes

# df_compras'forma_pagamento' - 700 valores nulos
# Preenchendo com "Não informado" para manter a consistência sem excluir linhas
df_compras['forma_pagamento'] = df_compras['forma_pagamento'].fillna('Não informado')

#  df_campanhas'tema_campanha' - 500 valores nulos
# Preenchendo com "Não informado" para evitar perda de dados em agrupamentos por tema
df_campanhas['tema_campanha'] = df_campanhas['tema_campanha'].fillna('Não informado')

# df_campanha'clicou' - 500 valores nulos
# Considerando que a ausência de clique para evitar perda da infrmação'
df_campanhas['clicou'] = df_campanhas['clicou'].fillna('não informado')

In [ ]:
# Checando se ainda restaram nulos nas duas bases

print("Nulos em df_compras:")
print(df_compras.isnull().sum())

df_campanhas.isnull().sum()

Nulos em df_compras:
cliente_id             0
data_compra            0
valor_compra           0
categoria              0
canal_compra           0
status_entrega         0
forma_pagamento        0
parcelado              0
quantidade_parcelas    0
dtype: int64


,0
cliente_id,0
data_envio,0
tema_campanha,0
canal,0
abriu,0
clicou,0
converteu_em_compra,0


##Analisando se tem duplicados

In [ ]:
#Verificar duplicatas
print("Duplicatas - CADASTRAL:", df_cadastral.duplicated().sum())
print("Duplicatas - CAMPANHAS:", df_campanhas.duplicated().sum())
print("Duplicatas - COMPRAS:", df_compras.duplicated().sum())

Duplicatas - CADASTRAL: 0
Duplicatas - CAMPANHAS: 0
Duplicatas - COMPRAS: 0


##Junção das Bases

Para entender melhor o perfil dos clientes que interagem com as campanhas de marketing, realizamos a junção entre duas tabelas:

Utilizamos a chave `cliente_id` para unir essas informações de forma precisa. A estratégia de junção foi o `left join`, preservando todos os registros de campanhas, mesmo que alguns clientes não tenham cadastro completo (caso raro, mas possível).

Com essa união, passamos a ter um único DataFrame consolidado `df_analise`, que será a base principal da nossa análise.

Depois a união das seguintes bases:

- `df_analise` (resultado da junção entre campanhas e dados cadastrais)
- `df_compras` (base contendo os registros de compras dos clientes)

A junção foi feita pela coluna `cliente_id`, utilizando `how='left'`, pois o objetivo é manter todos os clientes que receberam campanhas, mesmo que não tenham realizado compras. Isso é essencial para analisarmos o impacto das campanhas no comportamento de compra.

In [ ]:
# Unindo dados de campanhas com o perfil dos clientes
df_analise = pd.merge(df_campanhas, df_cadastral, on='cliente_id', how='left')

# Verificando as  primeiras linhas
df_analise.head(10)

,cliente_id,data_envio,tema_campanha,canal,abriu,clicou,converteu_em_compra,sexo,idade,faixa_etaria,...,cidade,escolaridade,profissao,estado_civil,renda_mensal,score_inicial,tipo_cliente,n_dependentes,canal_captacao,status_cadastro
0,C039242,2025-03-25,Compre e Ganhe,WhatsApp,Sim,Sim,Não,Feminino,27,25-34,...,Caruaru,Superior,Estudante,Solteiro(a),1500.00,475,Tradicional,1,Loja física,Inativo
1,C118819,2023-10-19,Cashback,SMS,Não,Não,Não,Masculino,47,45-59,...,Jaboatão dos Guararapes,Fundamental,Auxiliar de Serviços Gerais,Casado(a),1500.00,411,Tradicional,1,Loja física,Inativo
2,C114009,2024-04-01,Cashback,SMS,Sim,Não,Sim,Masculino,76,60+,...,Outros,Fundamental,Auxiliar de Serviços Gerais,Solteiro(a),1500.00,507,Potencial,2,Indicação,Bloqueado
3,C101599,2024-03-21,Desconto Exclusivo,Email,Sim,Sim,Sim,Feminino,38,35-44,...,Recife,Fundamental,Técnico de Enfermagem,Casado(a),1833.70,342,Tradicional,1,Loja física,Inativo
4,C088611,2024-12-27,Novo Produto,WhatsApp,Sim,Não,Não,Feminino,19,18-24,...,Outros,Fundamental,Servidor Público,Casado(a),2031.07,305,Tradicional,0,Loja física,Inativo
5,C065733,2024-08-21,Compre e Ganhe,Notificação App,Não,Não,Não,Masculino,40,35-44,...,Olinda,Médio,Autônomo,Solteiro(a),2214.92,429,Potencial,2,Campanha digital,Inativo
6,C140542,2024-10-10,Lançamento de Linha,Instagram Ads,Não,Não,Não,Masculino,20,18-24,...,Caruaru,Médio,Vendedor(a),Solteiro(a),1583.18,380,Digital,5,App,Inativo
7,C100687,2024-01-02,Lançamento de Linha,WhatsApp,Não,Sim,Não,Feminino,35,35-44,...,Paulista,Superior,Técnico de Enfermagem,Solteiro(a),3418.57,518,Tradicional,1,Loja física,Bloqueado
8,C018977,2023-08-31,Campanha de Recompra,SMS,Sim,Não,Não,Masculino,33,25-34,...,Camaragibe,Superior,Desempregado(a),Solteiro(a),1500.00,520,Tradicional,1,App,Bloqueado
9,C073789,2024-12-18,Desconto Exclusivo,WhatsApp,Não,Não,Sim,Feminino,33,25-34,...,Cabo de Santo Agostinho,Superior,Professor(a),Solteiro(a),2428.39,496,Tradicional,2,Loja física,Inativo


In [ ]:
# Unindo dados de compras com a base anterior (campanhas + cadastro dos clientes)
df_analise = pd.merge(df_analise, df_compras, on='cliente_id', how='left')

# Visualizando as primeiras linhas
df_analise.head(10)
df_analise.info()

#OBS: após aplicação do codigo, foi observado que o total de linhas foi 132.539, mesmo o base camapnha tendo apenas 100 mil
#o que em primeiro momento pareceu estranho, até observar que um mesmo cliente ID pode ter realizado mais de uma compra.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132539 entries, 0 to 132538
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   cliente_id           132539 non-null  object 
 1   data_envio           132539 non-null  object 
 2   tema_campanha        132539 non-null  object 
 3   canal                132539 non-null  object 
 4   abriu                132539 non-null  object 
 5   clicou               132539 non-null  object 
 6   converteu_em_compra  132539 non-null  object 
 7   sexo                 132539 non-null  object 
 8   idade                132539 non-null  int64  
 9   faixa_etaria         132539 non-null  object 
 10  estado               132539 non-null  object 
 11  cidade               132539 non-null  object 
 12  escolaridade         132539 non-null  object 
 13  profissao            132539 non-null  object 
 14  estado_civil         132539 non-null  object 
 15  renda_mensal     

##Análise Exploratória de Dados (EDA)

Nesta etapa, investigamos os dados combinados das bases Campanhas, Cadastral e Compras, buscando entender os padrões de comportamento dos clientes em relação às campanhas e compras.



###Distribuição dos perfis demográficos

In [ ]:
# Distribuição por sexo
print(df_analise['sexo'].value_counts(normalize=True) * 100)

# Distribuição por faixa etária
print(df_analise['faixa_etaria'].value_counts(normalize=True) * 100)

# Distribuição por escolaridade
print(df_analise['escolaridade'].value_counts())

sexo
Feminino     51.834554
Masculino    48.165446
Name: proportion, dtype: float64
faixa_etaria
25-34    28.153223
35-44    22.001071
45-59    19.855288
18-24    17.997721
60+      11.992696
Name: proportion, dtype: float64
escolaridade
Médio            55763
Superior         35403
Fundamental      33280
Pós-graduação     8093
Name: count, dtype: int64


In [ ]:
# Estatísticas da renda_mensal
df_analise['renda_mensal'] = pd.to_numeric(df_analise['renda_mensal'], errors='coerce')

df_analise['renda_mensal'].describe().round(2)

,renda_mensal
count,132539.00
mean,2370.47
std,834.86
min,1500.00
25%,1610.73
50%,2192.91
75%,2866.18
max,6594.42


##Engajamento campanha (abriu -> clicou -> comprou)


In [ ]:
# Taxa de abertura
print("Taxa de abertura de campanhas:")
print(df_analise['abriu'].value_counts(normalize=True) * 100)

# Taxa de clique
print("Taxa de clique nas campanhas:")
print(df_analise['clicou'].value_counts(normalize=True) * 100)

# Taxa de conversão
print("Taxa de conversão em compra:")
print(df_analise['converteu_em_compra'].value_counts(normalize=True) * 100)

Taxa de abertura de campanhas:
abriu
Sim    60.079675
Não    39.920325
Name: proportion, dtype: float64
Taxa de clique nas campanhas:
clicou
Não              69.651197
Sim              29.853100
não informado     0.495703
Name: proportion, dtype: float64
Taxa de conversão em compra:
converteu_em_compra
Não    82.013596
Sim    17.986404
Name: proportion, dtype: float64


## Criando um novo DF apenas com os que converteram em comprar, que é tema principal da nossa análise

In [ ]:
# Filtrando apenas os clientes que converteram em compra
df_convertidos = df_analise[df_analise['converteu_em_compra'] == 'Sim']

# Visualizando os primeiros registros para checar
display(df_convertidos.head(10))
df_convertidos.info()

#apareceu mais linhas do que os que so converteram em compras, pois alguns compraram mais de uma vez

,cliente_id,data_envio,tema_campanha,canal,abriu,clicou,converteu_em_compra,sexo,idade,faixa_etaria,...,canal_captacao,status_cadastro,data_compra,valor_compra,categoria,canal_compra,status_entrega,forma_pagamento,parcelado,quantidade_parcelas
3,C114009,2024-04-01,Cashback,SMS,Sim,Não,Sim,Masculino,76,60+,...,Indicação,Bloqueado,2025-02-11,185.44,Limpeza,Online,Cancelado,Pix,Não,1.0
4,C114009,2024-04-01,Cashback,SMS,Sim,Não,Sim,Masculino,76,60+,...,Indicação,Bloqueado,2024-10-24,100.70,Papelaria,Online,Atrasado,Pix,Sim,4.0
5,C114009,2024-04-01,Cashback,SMS,Sim,Não,Sim,Masculino,76,60+,...,Indicação,Bloqueado,2024-05-14,223.19,Alimentos,Online,Atrasado,Pix,Sim,4.0
6,C101599,2024-03-21,Desconto Exclusivo,Email,Sim,Sim,Sim,Feminino,38,35-44,...,Loja física,Inativo,2025-02-18,142.28,Limpeza,Online,Entregue,Débito,Sim,3.0
12,C073789,2024-12-18,Desconto Exclusivo,WhatsApp,Não,Não,Sim,Feminino,33,25-34,...,Loja física,Inativo,2023-09-25,534.44,Utilidades Domésticas,Aplicativo,Cancelado,Cartão,Sim,2.0
20,C118467,2024-05-16,Desconto Exclusivo,WhatsApp,Sim,Não,Sim,Feminino,30,25-34,...,App,Inativo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,C078258,2024-10-01,Compre e Ganhe,WhatsApp,Sim,Não,Sim,Masculino,53,45-59,...,Indicação,Inativo,2024-08-21,243.64,Limpeza,Online,Atrasado,Pix,Sim,4.0
29,C078258,2024-10-01,Compre e Ganhe,WhatsApp,Sim,Não,Sim,Masculino,53,45-59,...,Indicação,Inativo,2025-01-12,72.10,Alimentos,Presencial,Entregue,Boleto,Não,1.0
35,C109174,2024-07-29,Desconto Exclusivo,Notificação App,Sim,Não,Sim,Feminino,84,60+,...,Indicação,Inativo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,C086471,2024-06-10,Desconto Exclusivo,Instagram Ads,Sim,Não,Sim,Masculino,27,25-34,...,Indicação,Bloqueado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 23839 entries, 3 to 132531
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cliente_id           23839 non-null  object 
 1   data_envio           23839 non-null  object 
 2   tema_campanha        23839 non-null  object 
 3   canal                23839 non-null  object 
 4   abriu                23839 non-null  object 
 5   clicou               23839 non-null  object 
 6   converteu_em_compra  23839 non-null  object 
 7   sexo                 23839 non-null  object 
 8   idade                23839 non-null  int64  
 9   faixa_etaria         23839 non-null  object 
 10  estado               23839 non-null  object 
 11  cidade               23839 non-null  object 
 12  escolaridade         23839 non-null  object 
 13  profissao            23839 non-null  object 
 14  estado_civil         23839 non-null  object 
 15  renda_mensal         23839 non-null  flo

##Analisando o perfil só dos que converteram em compras

In [ ]:
# Distribuição por sexo
print(df_convertidos['sexo'].value_counts(normalize=True) * 100)

# Distribuição por faixa etária
print(df_convertidos['faixa_etaria'].value_counts(normalize=True) * 100)

# Distribuição por escolaridade
print(df_convertidos['escolaridade'].value_counts())

sexo
Feminino     52.208566
Masculino    47.791434
Name: proportion, dtype: float64
faixa_etaria
25-34    28.264608
35-44    21.674567
45-59    19.891774
18-24    17.978942
60+      12.190109
Name: proportion, dtype: float64
escolaridade
Médio            10231
Superior          6290
Fundamental       5908
Pós-graduação     1410
Name: count, dtype: int64


In [ ]:
#Estatísticas da renda_mensal
df_convertidos['renda_mensal'] = pd.to_numeric(df_analise['renda_mensal'], errors='coerce')

df_convertidos['renda_mensal'].describe().round(2)

/tmp/ipython-input-42-3357155995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_convertidos['renda_mensal'] = pd.to_numeric(df_analise['renda_mensal'], errors='coerce')


,renda_mensal
count,23839.00
mean,2380.01
std,839.37
min,1500.00
25%,1626.82
50%,2200.86
75%,2872.88
max,6594.42


##Verificando se houve mais de uma compra por cliente

In [ ]:
#Quantidade de vezes que cada cliente_id comprou
quantidade_por_cliente = df_convertidos.groupby('cliente_id').size().reset_index(name='quantidade_compras')
display(quantidade_por_cliente.head())


,cliente_id,quantidade_compras
0,C000019,2
1,C000030,1
2,C000032,2
3,C000034,1
4,C000040,1


##Média de quantas vezes cada cliente comprou

In [ ]:
# Contar quantas compras cada cliente fez
compras_por_cliente = df_convertidos['cliente_id'].value_counts()

# Calcular a média de compras por cliente
media_compras = compras_por_cliente.mean()

print(f"Média de compras por cliente: {media_compras:.2f}")


Média de compras por cliente: 1.40


##Quais canais e temas de campanha converteram mais em compras.

In [ ]:
# Agrupa por canal e tema_campanha, contando clientes únicos
clientes_por_grupo = df_analise.groupby(['canal', 'tema_campanha'])['cliente_id'].nunique().reset_index(name='qtd_clientes')

# Total geral de clientes únicos
total_clientes = df_analise['cliente_id'].nunique()

# Calcula o percentual
clientes_por_grupo['percentual'] = (clientes_por_grupo['qtd_clientes'] / total_clientes) * 100

# Arredonda para 2 casas decimais
clientes_por_grupo['percentual'] = clientes_por_grupo['percentual'].round(2)

# Exibe o resultado ordenado
clientes_por_grupo.sort_values(by='percentual', ascending=False, inplace=True)
display(clientes_por_grupo)



,canal,tema_campanha,qtd_clientes,percentual
31,WhatsApp,Desconto Exclusivo,7391,10.15
3,Email,Desconto Exclusivo,7304,10.03
17,Notificação App,Desconto Exclusivo,6032,8.29
24,SMS,Desconto Exclusivo,5781,7.94
29,WhatsApp,Cashback,4988,6.85
1,Email,Cashback,4906,6.74
22,SMS,Cashback,3975,5.46
15,Notificação App,Cashback,3746,5.15
33,WhatsApp,Novo Produto,3747,5.15
5,Email,Novo Produto,3643,5.00


In [ ]:
# Agrupar por categoria e contar clientes únicos
clientes_por_categoria = df_convertidos.groupby('categoria')['cliente_id'].nunique().reset_index(name='qtd_clientes')

# Calcular o total geral de clientes únicos que converteram
total_clientes_convertidos = df_convertidos['cliente_id'].nunique()

# Calcular o percentual de clientes que converteram por categoria
clientes_por_categoria['percentual'] = (
    clientes_por_categoria['qtd_clientes'] / total_clientes_convertidos * 100
).round(2)

# Exibir o resultado ordenado pelo percentual
clientes_por_categoria.sort_values(by='percentual', ascending=False, inplace=True)
display(clientes_por_categoria)

,categoria,qtd_clientes,percentual
0,Alimentos,4238,24.94
3,Higiene Pessoal,2208,12.99
4,Limpeza,2197,12.93
1,Bebidas,1764,10.38
2,Descartáveis,1471,8.66
6,Utilidades Domésticas,1454,8.56
5,Papelaria,1250,7.36


##Baixando o DF só com os clientes que converteram em compra para exortar para o  POWER BI

In [ ]:
# Exportando para CSV (sem o índice)
df_convertidos.to_csv('clientes_convertidos.csv', index=False)

# Gerando link para download no Colab
from google.colab import files
files.download('clientes_convertidos.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>